In [1]:
# import nstrace
import sys
import matplotlib.pyplot as plt
import matplotlib

In [2]:
filename = 'trace.tr'
with open(filename) as f:
    content = f.readlines()

In [3]:
splitter = lambda x: x.split()
split_list = list(map(splitter, content))

In [4]:
keep_cwnd = lambda x: x if 'cwnd_' in x else None
just_cwnd = list(map(keep_cwnd, split_list))

In [5]:
just_cwnd = list(filter((None).__ne__, just_cwnd))

In [6]:
data_maker = lambda x: [x[0], x[1], x[3], x[6]]
data = list(map(data_maker, just_cwnd))

In [7]:
stream_1_maker = lambda x: x if x[1] == '0' and x[2] == '4' else None
stream_1 = list(map(stream_1_maker, data))
stream_1 = list(filter((None).__ne__, stream_1))

stream_2_maker = lambda x: x if x[1] == '1' and x[2] == '5' else None
stream_2 = list(map(stream_2_maker, data))
stream_2 = list(filter((None).__ne__, stream_2))

In [8]:
print(len(stream_1), len(stream_2))

467 476


***

In [9]:
cwnd_file = 'cwnd'
with open(cwnd_file) as f:
    content = f.readlines()
splitter = lambda x: x[:-1].split()
split_cwnd = list(map(splitter, content))

In [10]:
time = list(map(lambda x: x[0], split_cwnd))
cwnd1 = list(map(lambda x: x[1], split_cwnd))
cwnd2 = list(map(lambda x: x[2], split_cwnd))

In [12]:
# plt.plot(time,cwnd1)
# matplotlib.use('TkAgg')
plt.figure(figsize=(30,15))
plt.plot(time, cwnd1)
plt.plot(time, cwnd2)
plt.legend(['cwnd 1 to 5 tcp reno', 'cwnd 2 to 6 tcp reno'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(30,15))
plt.plot(time, cwnd1, color='green')

***

In [37]:
filename = 'trace.tr'
with open(filename) as f:
    content = f.readlines()
split_rtt = list(map(lambda x: x[:-1].split(), content))
rtt_data = list(map(lambda x: x if 'rtt_' in x else None, split_rtt))
rtt_data = list(filter((None).__ne__, rtt_data))

In [40]:
rtt_1 = list(map(lambda x: [x[0],x[6]] if x[1] == '0' and x[2] == '4' else None, rtt_data))
rtt_1 = list(filter((None).__ne__, rtt_1))

rtt_2 = list(map(lambda x: [x[0],x[6]] if x[1] == '1' and x[2] == '5' else None, rtt_data))
rtt_2 = list(filter((None).__ne__, rtt_2))

rtt1_time = list(map(lambda x: x[0], rtt_1))
rtt1_value = list(map(lambda x: x[1], rtt_1))

rtt2_time = list(map(lambda x: x[0], rtt_2))
rtt2_value = list(map(lambda x: x[1], rtt_2))

In [41]:
matplotlib.use('TkAgg')
plt.figure(figsize=(30,15))
plt.plot(rtt1_time, rtt1_value)
plt.plot(rtt2_time, rtt2_value)
plt.legend(['rtt 1 to 5 tcp reno', 'rtt 2 to 6 tcp reno'], loc='upper left')
plt.show()